In [11]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd

I download the Wiki page and rea it with BeautifoulSoup

In [12]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
soup = BeautifulSoup(response.text,'html.parser')

In [13]:
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [14]:
rows = table.find_all('tr')

In [15]:
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

In [16]:
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


I create the dataframe with the postal codes.
The last line of the cell is the onluy way I found to skip di "\n" from the Neighbourhood

In [18]:
df = pd.DataFrame(columns=columns)

In [19]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 4:
        values = [tds[0].text, tds[1].text,'', tds[2].text.replace('\n',''), tds[3].replace('\n','')]
    else:
        values = [td.text.replace('\n','') for td in tds]
  

    df = df.append(pd.Series(values, index = columns), ignore_index=True)
    df['Neighbourhood'].replace('\n','')
    

In [20]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


I drop the lines where the Borough value is "Not assigned"

In [21]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [22]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


I group all the neighbourhoods with the same Borough

In [23]:
df1 = df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [24]:
df1.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


Finally I assign the value in the Borough cell to Neighbouhoods with "Not assigned" value

In [25]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

I check if in the 9th cell in the table the value of the Borough and of the Neighborhood columns are both "Queen's Park"

Finally, I check the shape of the dataframe

In [31]:
df1.shape

(103, 2)

I download the csv file with the Postal codes in a dataframe

In [32]:
coord = pd.read_csv('geospatial_coordinates.csv')

I rename the Postal Code columns to allow the merge with the other database

In [34]:
coord = coord.rename(columns = {'Postal Code':'Postcode'})

In [38]:
df1_new = pd.merge(df1, coord,  how = 'outer', on='Postcode')

Here is the new dataframe

In [39]:
df1_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [40]:
df1_new.shape

(103, 5)